In [1]:
import sys
import os
from typing import Tuple
import pandas as pd 
import mlflow as mf

## replace with your specific paths
sys.path.append("/home/aamir07/mlops2_with_dagster/") 
ARTIFACT_PATH = "/home/aamir07/mlops2_with_dagster/artifacts/mlflow_artfacts" 
LOGGER_FOLDER_PATH = "/home/aamir07/mlops2_with_dagster/notebooks"

from mlops2_with_dagster import encoder_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse
from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)

from ortho import BaseKernel
from ortho.utils import Logger
from ortho.ortho.decorators import task
from ortho.ortho.callbacks import MlFlowCallBack, LoggerCallBack


logger = Logger().logger()

   

        
        

None


2023-09-15 00:10:01.518432: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:10:01.982912: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:10:01.985855: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 00:10:04,965 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-15 00:10:05.185155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-15 00:10:05,327 - DEBUG - Creating converter from 7 to 5
2023-09-15 00:10:05,329 - DEBUG - Creating converter from 5 to 7
2

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/aamir07/mlops2_with_dagster


In [3]:
class Encoder(BaseKernel):
    
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        super().__init__(callbacks,
                       experiment_name,
                       run_name,
                       load_from_artifact)
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
    
    @task()       
    def create_encoders(self, df_train, df_test):
        adapter = base.SimplePythonGraphAdapter(base.DictResult())
        encode_dr = driver.Driver(self.config, encoder_pipeline, adapter=adapter)
        output_nodes = ['encoders']
        
        out = encode_dr.execute(output_nodes ,
        inputs = dict(
            df_train = df_train,
            df_test = df_test
        )         
        )
        
        data = {"encoders": out }
        payload = {"params":self.config,
                   "artifact_path": ARTIFACT_PATH}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.create_encoders(*args, **kwargs)
        
        
    

In [4]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [5]:
# parameters
df_train = pd.read_csv(train_data)
df_test = pd.read_csv(test_data)

In [6]:
import dagstermill
encoders_object = Encoder(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack(log_folder=LOGGER_FOLDER_PATH,
                                                      kernel_names=["create_encoders"]),
                                       MlFlowCallBack(kernel_names=["create_encoders"])])

mf.end_run()
dagstermill.yield_result(encoders_object.run(df_train=df_train,
                                            df_test=df_test)[0]["encoders"], output_name="encoders")

2023-09-15 00:10:09,055 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002
2023-09-15 00:10:09,097 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-09-15 00:10:09,379 - DEBUG - Popen(['git', 'version'], cwd=/home/aamir07/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-09-15 00:10:09,410 - DEBUG - Popen(['git', 'version'], cwd=/home/aamir07/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-09-15 00:10:09,428 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:10:09,495 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 914
2023-09-15 00:10:09,499 - INFO - Successfuly initiated run with name BadamBhum and id a05f805113ff42a3b54371547a26d49b
2023-09-15 00:10:09,502 - WARNING - Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over 

{'encoders': {'cabinencoder': LabelEncoder(),
  'sexencoder': LabelEncoder(),
  'embarkedencoder': LabelEncoder()}}

2023-09-15 00:10:09,928 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35", "python_version": "3.10.11/CPython", "distinct_id": "f41b7308-274c-4333-ab85-eca6fe911802", "hamilton_version": [1, 28, 0], "telemetry_version": "0.0.1", "number_of_nodes": 32, "number_of_modules": 1, "number_of_config_items": 3, "decorators_used": {"parameterize_sources": 4, "extract_columns": 2, "extract_fields": 1}, "graph_adapter_used": "hamilton.base.SimplePythonGraphAdapter", "result_builder_used": "hamilton.base.DictResult", "driver_run_id": "a9bcef00-5a5e-471c-93f5-e5db41ce5063", "error": null, "graph_executor_class": "DefaultGraphExecutor"}}'].
2023-09-15 00:10:09,968 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event